In [3]:
import requests
import json

GROQCLOUD_API_KEY = "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"  # Replace with your actual key
GROQCLOUD_EMBEDDING_ENDPOINT = "https://api.groqcloud.com/embeddings" # Replace with the actual endpoint
GROQCLOUD_EMBEDDING_MODEL = "llama-3.3-70b-versatile" # Replace with the model name

def get_groq_embeddings(texts):
    headers = {
        "Authorization": f"Bearer {GROQCLOUD_API_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": GROQCLOUD_EMBEDDING_MODEL,
        "input": texts
    }
    try:
        response = requests.post(GROQCLOUD_EMBEDDING_ENDPOINT, headers=headers, data=json.dumps(payload))
        response.raise_for_status()  # Raise an exception for bad status codes
        data = response.json()
        return [embedding["embedding"] for embedding in data["data"]] # Adjust based on API response structure
    except requests.exceptions.RequestException as e:
        print(f"Error getting embeddings from GroqCloud: {e}")
        return None

documents = [
    "The weather today is pleasant in Nawabshah.",
    "GroqCloud offers fast AI inference.",
    "Summer can be very hot.",
    "Python is great for data science."
]

embeddings = get_groq_embeddings(documents)

if embeddings:
    print("Successfully obtained embeddings from GroqCloud.")
    # Now you can proceed to add these embeddings to ChromaDB

Error getting embeddings from GroqCloud: HTTPSConnectionPool(host='api.groqcloud.com', port=443): Max retries exceeded with url: /embeddings (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x00000217BE7BAC90>: Failed to resolve 'api.groqcloud.com' ([Errno 11001] getaddrinfo failed)"))


In [ ]:
import chromadb

# Assuming you have your ChromaDB client and collection initialized
client = chromadb.Client()
collection_name = "groq_collection"
collection = client.get_or_create_collection(name=collection_name)

documents = [
    "The weather today is pleasant in Nawabshah.",
    "GroqCloud offers fast AI inference.",
    "Summer can be very hot.",
    "Python is great for data science."
]
metadatas = [
    {"location": "Nawabshah", "topic": "weather"},
    {"company": "Groq", "topic": "AI"},
    {"location": "Nawabshah", "topic": "weather", "season": "summer"},
    {"language": "Python", "topic": "programming"}
]
ids = ["doc1", "doc2", "doc3", "doc4"]

# Assuming 'embeddings' is the list of embeddings you got from GroqCloud
if embeddings and len(embeddings) == len(documents):
    collection.add(
        embeddings=embeddings,
        documents=documents,
        metadatas=metadatas,
        ids=ids
    )
    print(f"Added {collection.count()} items to the GroqCloud-backed collection.")
else:
    print("Error: Number of embeddings does not match the number of documents.")